## Creating Visualization Datasets

Now that we have explored the data a bit. Let's create our visualized datasets. 

Receiving Stats from each dataset:

1. Next Gen Stats (NGS):
    1. 'week'
    2. 'player_display_name',
    3. 'player_position'
    4. 'avg_cushion'
    5. 'avg_separation'
    6. 'avg_intended_air_yards'
    7. 'percent_share_of_intended_air_yards'
    8. 'receptions'
    9. 'targets'
    10. 'catch_percentage'
    11. 'yards'
    12. 'rec_touchdowns'
    13. 'avg_yac'
    14. 'avg_expected_yac'
    15. 'player_gsis_id'

2. Pro Football Reference (PFR):
    1. 'game_id'
    2. 'pfr_player_id',
    3. 'receiving_broken_tackles'
    4. 'receiving_drop'
    5. 'receiving_drop_pct'
    6. 'receiving_int'
    7. 'receiving_rat'

3. Weekly (NFL?)
    1. 'player_id'
    2. 'position'
    3. 'position_group'
    4. 'headshot_url'
    5. 'week'
    6. 'receptions' *for checking*
    7. 'targets' *for checking*
    8. 'receiving_yards' *for checking*
    9. 'receiving_tds' *for checking*
    10. 'receiving_fumbles'
    11. 'receiving_fumbles_lost'
    12. 'receiving_air_yards' *for checking*
    13. 'receiving_yards_after_catch' *for checking*
    14. 'receiving_first_downs'
    15. 'receiving_epa'
    16. 'receiving_2pt_conversions'
    17. 'racr'
    18. 'target_share'
    19. 'air_yards_share'
    20. 'wopr'
    21. 'fantasy_points'
    22. 'fantasy_points_ppr'
    23. 'game_id'

New stats to synthesize:
1. Garbage time targets/yards
2. Redzone targets

Play-by-Play Data To synthesize new stats

1. ftn = nfl.import_ftn_data(years = [2023])
2. pbp_data = nfl.import_pbp_data(years = [2023])

links:
1. ESPN API: https://github.com/rbarton65/espnff
2. ESPN API: https://github.com/cwendt94/espn-api
3. NFL DATA API: https://github.com/cooperdff/nfl_data_py/tree/main
4. Plotting and Dashboarding with Plotly and Dash: https://www.youtube.com/watch?v=XOFrvzWFM7Y

In [1]:
import nfl_data_py as nfl
import pandas as pd
# import seaborn as sns
# import matlotlib.pyplot plt
year = 2022

In [2]:
#player info
id_map = nfl.import_ids()
id_map['gsis_id'] = id_map['gsis_id'].replace({'00-0034439': '00-0034270', '00-0034270':'11-1111111'})
id_map['team'] = id_map['team'].replace({
    'LVR':'LV',
    'KCC':'KC',
    'NOS':'NO',
    'TBB':'TB',
    'SFO':'SF',
    'NEP':'NE',
    'LAR':'LA',
    'GBP':'GB',
    'JAC':'JAX'
})

In [3]:
# depth charts
depth_charts = nfl.import_depth_charts(years = [year])
depth_charts = depth_charts[depth_charts['formation'] == "Offense"]
# importing next gen stats data
ngs = nfl.import_ngs_data(stat_type = 'receiving', years = [year]).drop_duplicates()[['week',
        'team_abbr',
        'player_display_name',
        'player_position',
        'avg_cushion',
        'avg_separation',
        'avg_intended_air_yards',
        'percent_share_of_intended_air_yards',
        'receptions',
        'targets',
        'catch_percentage',
        'yards',
        'rec_touchdowns',
        'avg_yac',
        'avg_expected_yac',
        'player_gsis_id']]

# pro football reference stats
pfr = nfl.import_weekly_pfr(s_type = "rec", years = [year]).drop_duplicates()[['game_id',
                        'pfr_player_id',
                        'receiving_broken_tackles',
                        'receiving_drop',
                        'receiving_drop_pct',
                        'receiving_int',
                        'receiving_rat']]

# # weekly aggregate stats
# weekly = nfl.import_weekly_data(years = [year], downcast=True)[[
#     'player_id',
#     'opponent_team',
#     'position',
#     'position_group',
#     'headshot_url',
#     'week',
#     'receptions',
#     'targets',
#     'receiving_yards',
#     'receiving_tds',
#     'receiving_fumbles',
#     'receiving_fumbles_lost',
#     'receiving_air_yards',
#     'receiving_yards_after_catch',
#     'receiving_first_downs',
#     'receiving_epa',
#     'receiving_2pt_conversions',
#     'racr',
#     'target_share',
#     'air_yards_share',
#     'wopr',
#     'fantasy_points',
#     'fantasy_points_ppr'
# ]].drop_duplicates()

In [4]:
# I need to standardize the game ID and player ID's accross the dataframes
matchup_data = nfl.import_schedules([year])
game_id_map = pd.concat([matchup_data[['game_id', 'home_team', 'week']].rename({'home_team':'team_abbr'}, axis=1), matchup_data[['game_id', 'away_team', 'week']].rename({'away_team':'team_abbr'}, axis=1)], ignore_index=False)

In [5]:
# grab game_id for ngs/weekly datasets
ngs = ngs.merge(game_id_map, left_on=['week', 'team_abbr'], right_on = ['week', 'team_abbr']).rename({'player_gsis_id':'player_id'},axis=1)
# weekly = weekly.merge(game_id_map, left_on=['week', 'opponent_team'], right_on = ['week', 'team_abbr'])
pfr = pfr.merge(id_map[['pfr_id', 'gsis_id']], left_on='pfr_player_id', right_on = 'pfr_id').rename({'gsis_id':'player_id'},axis=1)
# ngs data needs player ID as well

# depth charts merge
depth_charts = depth_charts.merge(game_id_map, left_on=['week', 'club_code'], right_on = ['week', 'team_abbr'])

### looks like NGS doesn't have receiving data for RB's. What can I get from the pbp data?
1. avg_intended_air_yards
2. percentage_share_of_intended_air_yards
3. receptions
4. targets
5. catch_percentage
6. Yards
7. rec_touchdowns
8. avg_yac
9. avg_expected_yac

avg_cushion is the only one I'm not sure about.  We can have cushion/separation stats just for TE's and WR's


In [6]:
# grab play-by-play data to synthesize some base stats. Adding in stats from other platforms after the fact.
pbp_data = nfl.import_pbp_data(years = [year])

2022 done.
Downcasting floats.


In [8]:
[x for x in pbp_data.columns if 'fumble' in x.lower()]

['fumble_forced',
 'fumble_not_forced',
 'fumble_out_of_bounds',
 'fumble_lost',
 'fumble',
 'forced_fumble_player_1_team',
 'forced_fumble_player_1_player_id',
 'forced_fumble_player_1_player_name',
 'forced_fumble_player_2_team',
 'forced_fumble_player_2_player_id',
 'forced_fumble_player_2_player_name',
 'fumbled_1_team',
 'fumbled_1_player_id',
 'fumbled_1_player_name',
 'fumbled_2_player_id',
 'fumbled_2_player_name',
 'fumbled_2_team',
 'fumble_recovery_1_team',
 'fumble_recovery_1_yards',
 'fumble_recovery_1_player_id',
 'fumble_recovery_1_player_name',
 'fumble_recovery_2_team',
 'fumble_recovery_2_yards',
 'fumble_recovery_2_player_id',
 'fumble_recovery_2_player_name']

In [10]:
pbp_data[pbp_data['fumble'] == 1]['fumbled_2_player_id'].isna().sum()

666

In [13]:
# let's see if we can get a fumble lost player id
def get_fumble_lost_receiver_id(x):
    rpid = x['receiver_player_id']
    offense = x['posteam']
    # if there was a fumble and the receiver we are looking at was one of the fumble players
    if x['fumble_lost'] == 1 and (x['fumbled_1_player_id'] == rpid or x['fumbled_2_player_id'] == rpid):
        # the only time this is not a fumble for the player in consideration is when
        # there are two fumbles by the offense and the second player is not the first player.
        # e.g. player 1 fumbled and it was picked up by teammate player 2, then player 2 fumbles
        # and it is recovered by the defense
        if (x['fumbled_1_player_id'] == rpid and x['fumbled_2_player_id'] != rpid) and \
            (x['fumbled_1_team'] == offense and x['fumbled_2_team'] == offense):
            return 0
        else:
            return 1
    else:
        return 0
    
pbp_data['receiver_fumble_lost'] = pbp_data.apply(get_fumble_lost_receiver_id, axis=1)

129      (14:13) (No Huddle, Shotgun) 19-J.Flacco pass ...
216      (13:36) (Shotgun) 28-J.Cook right tackle to BU...
321      (5:27) 20-Z.Moss left tackle to LA 16 for 4 ya...
621      (7:42) (Shotgun) 25-M.Gordon right tackle to S...
629      (5:41) (No Huddle, Shotgun) 7-G.Smith pass sho...
                               ...                        
49758    (1:00) (Shotgun) 15-P.Mahomes sacked at KC 49 ...
49832    (7:03) 13-B.Purdy pass incomplete short left t...
49890    (1:19) (No Huddle, Shotgun) 17-J.Johnson FUMBL...
49960    (4:13) 19-D.Samuel left end to SF 31 for -7 ya...
50023    (9:48) (Shotgun) 1-J.Hurts up the middle to PH...
Name: desc, Length: 293, dtype: object

In [14]:
pbp_data[pbp_data['receiver_fumble_lost'] == 1]['desc'].iloc[0]

'(14:13) (No Huddle, Shotgun) 19-J.Flacco pass short middle to 20-Br.Hall to BAL 16 for 6 yards (36-C.Clark). FUMBLES (36-C.Clark), RECOVERED by BAL-44-M.Humphrey at BAL 15.'

In [7]:
d = {}
d['is_two_point_conversion'] = pbp_data['two_point_conv_result'].apply(lambda x: 1 if x == 'success' else 0)
pbp_data = pd.concat([pbp_data, pd.DataFrame(d)], axis=1)
pbp_data['touchdown'] = pbp_data['touchdown'].fillna(0)
pbp_data['offense_players'] = pbp_data['offense_players'].apply(lambda x: x.split(';'))

export = False
if export:
    pbp_data.to_excel(f"./test_{year}_nfl_data.xlsx", index=False)

In [8]:
#FIXME: This is not average depth of target. It is average depth of reception. Max target depth as well.
# pbp_data = pbp_data[['air_yards', 'receiving_yards', 'yards_after_catch']].fillna(0)
agg_WR = pbp_data[(pbp_data['complete_pass'] == 1) & (pbp_data['play_type'] != 'no_play')].groupby(['receiver_player_id', 'receiver_player_name', 'game_id']).agg(
    receiving_yards = ('receiving_yards', 'sum'),
    avg_yac = ('yards_after_catch', 'mean'),
    # avg_depth_of_target = ('air_yards', 'mean'),
    receptions = ('complete_pass', 'count'),
    # max_target_depth = ('air_yards', 'max'),
    receiving_touchdowns = ('touchdown', 'sum'), 
    week = ('week', 'first')
).reset_index()

In [9]:
agg_WR[(agg_WR['receiver_player_id'] == '00-0030035') & (agg_WR['game_id'] == '2023_02_NO_CAR')]

,receiver_player_id,receiver_player_name,game_id,receiving_yards,avg_yac,receptions,receiving_touchdowns,week


In [10]:
# FIXME: For players who were not injured, but did not receive any of the above stats, I need to insert rows here.

rosters = nfl.import_weekly_rosters(years=[year])[['week', 'position', 'player_name', 'player_id','team', 'status']].merge(game_id_map, left_on=['week', 'team'], right_on = ['week', 'team_abbr']).drop('team_abbr', axis=1)
# get game id
rosters = rosters[rosters['status'] == 'ACT'].drop('status',axis=1)

for _, player in rosters[rosters['position'].isin(['WR', 'RB', 'FB', 'TE', 'QB'])].iterrows():
    if agg_WR[(agg_WR.receiver_player_id.isin([player.player_id])) & (agg_WR.week.isin([player.week]))].empty:
        agg_WR = pd.concat([agg_WR, pd.DataFrame([[player.player_id, None, player.game_id, 0, 0, 0, 0, player.week]], columns = agg_WR.columns)], ignore_index=True)

In [11]:
agg_WR[(agg_WR['receiver_player_id'] == '00-0030035') & (agg_WR['game_id'] == '2023_02_NO_CAR')]

,receiver_player_id,receiver_player_name,game_id,receiving_yards,avg_yac,receptions,receiving_touchdowns,week


In [12]:
# number of targets per game
targets_game = (
    pbp_data[(pbp_data["pass_attempt"] == 1) & (pbp_data["down"].isin([1, 2, 3, 4]))]
    .groupby(["receiver_player_id", "game_id", "down"])["pass_attempt"]
    .count()
    .reset_index()
    .rename({"pass_attempt": "targets"}, axis=1)
)
targets_game = (
    targets_game.pivot(
        index=["game_id", "receiver_player_id"], columns=["down"], values="targets"
    )
    .reset_index()
    .fillna(0)
    .rename({1.0: "targets_1", 2.0: "targets_2", 3.0: "targets_3", 4.0: "targets_4"}, axis=1)
)
targets_game["total_targets"] = (
    targets_game["targets_1"]
    + targets_game["targets_2"]
    + targets_game["targets_3"]
    + targets_game["targets_4"]
)

team_targets_game = (
    pbp_data[(pbp_data["pass_attempt"] == 1) & (pbp_data["down"].isin([1, 2, 3, 4]))]
    .groupby(["game_id", "posteam", "down"])["pass_attempt"]
    .count()
    .reset_index()
    .rename({"pass_attempt": "targets"}, axis=1)
)
team_targets_game = (
    team_targets_game.pivot(
        index=["game_id", "posteam"], columns=["down"], values="targets"
    )
    .reset_index()
    .fillna(0)
    .rename({"posteam":"team", 1.0: "ttargets_1", 2.0: "ttargets_2", 3.0: "ttargets_3", 4.0: "ttargets_4"}, axis=1)
)
team_targets_game["ttotal_targets"] = (
    team_targets_game["ttargets_1"]
    + team_targets_game["ttargets_2"]
    + team_targets_game["ttargets_3"]
    + team_targets_game["ttargets_4"]
)

air_yards_game = (
    pbp_data[(pbp_data["pass_attempt"] == 1)]
    .groupby(["receiver_player_id", "game_id"])
    .agg(
        avg_depth_of_target = ('air_yards', 'mean'),
        air_yards = ('air_yards', 'sum'),
        max_target_depth = ('air_yards', 'max')
    )
    .reset_index()
)

team_air_yards_game = (
    pbp_data[(pbp_data["pass_attempt"] == 1)]
    .groupby(["game_id", "posteam"])
    .agg(
        team_air_yards = ('air_yards', 'sum')
    )
    .reset_index()
    .rename({'posteam':'team'}, axis=1)
)


# num_snaps is a bit more complicated. For each player, we need to count the number of times their player ID appeared in the offensive snaps,
# then we can count the number of offensive snaps and get the percentage from that.
# snaps = nfl.import_snap_counts(years = [year]).merge(id_map, left_on='pfr_player_id', right_on = 'pfr_id')

# redzone targets
redzone_targets_game = (
    pbp_data[
        (pbp_data["pass_attempt"] == 1)
        & (pbp_data["yardline_100"] <= 20)
        & (pbp_data["play_type"] != "no_play")
    ]
    .groupby(["receiver_player_id", "game_id"])["pass_attempt"]
    .count()
    .reset_index()
    .rename({"pass_attempt": "rz_targets"}, axis=1)
)

# garbage time points?
# fantasy points scored when the point differential is more than 2 scores with 10 or less minutes left in the game
# TODO: Address the definition here. Should the point total be higher? What if they're down by 4 scores with a quarter left?
grouped = pbp_data[
    (pbp_data["play_type"] != "no_play")
    & (
        (
            (pbp_data["complete_pass"] == 1)
            & (pbp_data["score_differential"] <= -28)
            & (pbp_data["game_seconds_remaining"] <= 15 * 60)
        )
        | (
            (pbp_data["complete_pass"] == 1)
            & (pbp_data["score_differential"] <= -21)
            & (pbp_data["game_seconds_remaining"] <= 10 * 60)
        )
        | (
            (pbp_data["complete_pass"] == 1)
            & (pbp_data["score_differential"] <= -14)
            & (pbp_data["game_seconds_remaining"] <= 3 * 60)
        )  # ((pbp_data['complete_pass'] == 1) & (pbp_data['score_differential'] < -10) & (pbp_data['game_seconds_remaining'] <= 2*60)) \
    )
].groupby(["receiver_player_id", "game_id"])


def get_gtfp(x):
    d = {}
    d["garbage_time_fpoints"] = (
        0.1 * x["yards_gained"] + 1 + 6 * x["touchdown"] + 2 * x["is_two_point_conversion"]
    ).sum()
    return pd.Series(d, index=["garbage_time_fpoints"])


gtfp = grouped.apply(get_gtfp).reset_index()

# fantasy points per game (PPR)
grouped2 = pbp_data[
    (pbp_data["complete_pass"] == 1) & (pbp_data["play_type"] != "no_play")
].groupby(["receiver_player_id", "game_id"])


def get_fp(x):
    d = {}
    d["receiving_fpoints"] = (
        0.1 * x["yards_gained"]
        + 1
        + 6 * x["touchdown"]
        + 2 * x["is_two_point_conversion"]
    ).sum()
    return pd.Series(d, index=["receiving_fpoints"])


rfp = grouped2.apply(get_fp).reset_index()

In [13]:
# what I want here

# 1. Total offensive snaps (non-kickoff/kick/punt)
# 2. Total number of 1st, 2nd, 3rd, 4th downs
# 3. Total number of 1st, 2nd, 3rd, 4th downs per player

# player snaps
temp_snap_db = (
    pbp_data[
        (~pbp_data["play_type"].isin(["kickoff", "extra_point", "field_goal", "punt"]))
        & (~pbp_data["desc"].str.contains("Punt formation"))
        & (~pbp_data["desc"].str.contains(" punts "))
        & (~pbp_data["desc"].str.contains("Kick formation"))
    ][["game_id", "down", "offense_players", "play_id"]]
    .dropna(subset=["down"])
    .explode("offense_players")
)
# NOTE: We left in no_play's because we want to know if receivers are lining up before timeouts/penalty calls
player_snaps = (
    temp_snap_db.groupby(["game_id", "down", "offense_players"])["play_id"]
    .count()
    .reset_index()
)
player_snaps = (
    player_snaps.pivot(
        index=["game_id", "offense_players"], columns=["down"], values="play_id"
    )
    .reset_index()
    .fillna(0)
)
player_snaps = player_snaps.rename(
    {
        "offense_players": "receiver_player_id",
        1.0: "snap_count_1",
        2.0: "snap_count_2",
        3.0: "snap_count_3",
        4.0: "snap_count_4",
    },
    axis=1,
)
player_snaps["total_relevant_snaps"] = (
    player_snaps["snap_count_1"]
    + player_snaps["snap_count_2"]
    + player_snaps["snap_count_3"]
    + player_snaps["snap_count_4"]
)

# total offense snaps
temp_team_snap_db = pbp_data[
    (~pbp_data["play_type"].isin(["kickoff", "extra_point", "field_goal", "punt"]))
    & (~pbp_data["desc"].str.contains("Punt formation"))
    & (~pbp_data["desc"].str.contains(" punts "))
    & (~pbp_data["desc"].str.contains("Kick formation"))
][["game_id", "down", "play_id", "posteam"]].dropna(subset=["down"])
team_snaps = (
    temp_team_snap_db.groupby(["game_id", "down", "posteam"])["play_id"]
    .count()
    .reset_index()
)
team_snaps = (
    team_snaps.pivot(index=["game_id", "posteam"], columns=["down"], values="play_id")
    .reset_index()
    .fillna(0)
)
team_snaps = team_snaps.rename(
    {
        'posteam':'team',
        1.0: "tsnap_count_1",
        2.0: "tsnap_count_2",
        3.0: "tsnap_count_3",
        4.0: "tsnap_count_4",
    },
    axis=1,
)
team_snaps["ttotal_relevant_snaps"] = (
    team_snaps["tsnap_count_1"]
    + team_snaps["tsnap_count_2"]
    + team_snaps["tsnap_count_3"]
    + team_snaps["tsnap_count_4"]
)

In [14]:
# receiving fumbles
rec_fumbles = pbp_data[
    (pbp_data["complete_pass"] == 1) & (pbp_data["fumble"] == 1)
].groupby(["receiver_player_id", "game_id"])['fumble'].sum().reset_index().rename({'fumble':'receiving_fumbles'}, axis=1)

# receiving fumbles lost
rec_fumbles_lost = pbp_data[
    (pbp_data["complete_pass"] == 1) & (pbp_data["fumble_lost"] == 1)
].groupby(["receiver_player_id", "game_id"])['fumble_lost'].sum().reset_index().rename({'fumble_lost':'receiving_fumbles_lost'}, axis=1)

# number of first downs
rec_first_downs = pbp_data[
    (pbp_data["complete_pass"] == 1) & (pbp_data["first_down"] == 1)
].groupby(["receiver_player_id", "game_id"])['first_down'].count().reset_index().rename({'first_down':'receiving_first_downs'}, axis=1)

# per game epa
rec_epa = pbp_data[
    (pbp_data["pass_attempt"] == 1)
].groupby(["receiver_player_id", "game_id"])['epa'].sum().reset_index().rename({'epa':'receiving_epa'}, axis=1)

# number of 2 point conversions
rec_2ptconv = pbp_data[
    (pbp_data["complete_pass"] == 1) & (pbp_data["is_two_point_conversion"] == 1)
].groupby(["receiver_player_id", "game_id"])['is_two_point_conversion'].count().reset_index().rename({'is_two_point_conversion':'receiving_2pt_conversions'}, axis=1)

# number of intended air yards not resulting in a reception
unrealized_air_yards = pbp_data[
    (pbp_data["pass_attempt"] == 1) & (pbp_data["complete_pass"] == 0)
].groupby(["receiver_player_id", "game_id"])['air_yards'].sum().reset_index().rename({'air_yards':'unrealized_air_yards'}, axis=1)

# racr (ratio of receiving yards to air yards)
racr = pbp_data[
    (pbp_data["pass_attempt"] == 1)
].groupby(["receiver_player_id", "game_id"]).agg({'receiving_yards':'sum', 'air_yards':'sum'}).reset_index()
racr["racr"] = racr['receiving_yards']/racr['air_yards']
racr = racr.drop(['receiving_yards', 'air_yards'], axis=1)
racr = racr.dropna(subset=['racr'])





In [15]:
# merge all this together
agg_WR = agg_WR.merge(targets_game, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(redzone_targets_game, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(gtfp, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(rfp, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(air_yards_game, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(rec_fumbles, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(rec_fumbles_lost, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(rec_first_downs, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(rec_epa, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(rec_2ptconv, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(unrealized_air_yards, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(racr, on=['game_id', 'receiver_player_id'], how='outer')
agg_WR = agg_WR.merge(player_snaps, on=['game_id', 'receiver_player_id'], how='left').fillna(0).rename({'receiver_player_id':'player_id'},axis=1)
# agg_WR['player_id'] = agg_WR['player_id'].replace({'00-0034270':'00-0034439'})
# FIXME: There's something going on with this ID. seems to be an issue with conklin

In [16]:
agg_WR = agg_WR.merge(
    pfr[['game_id',
        'player_id',
        'receiving_broken_tackles',
        'receiving_drop',
        'receiving_drop_pct',
        'receiving_int',
        'receiving_rat']],
    on=["game_id", "player_id"],
    how="left",
)
print(f"Post PFR merge: {len(agg_WR)}")

Post PFR merge: 7899


In [17]:
agg_WR = agg_WR.merge(
    ngs[['game_id',
        'player_id',
        'avg_cushion',
        'avg_separation']],
    on=["game_id", "player_id"],
    how="left",
)
print(f"Post NGS merge: {len(agg_WR)}")

Post NGS merge: 7899


In [18]:

# agg_WR_final = agg_WR.drop('receiver_player_name',axis=1).merge(id_map[['gsis_id', 'name', 'position', 'team', 'birthdate', 'age',
#                                                             'draft_year', 'draft_round', 'draft_pick', 'draft_ovr',
#                                                             'twitter_username', 'height', 'weight', 'college']].rename({'gsis_id':"player_id"},axis=1),
#                                                               on = "player_id", how="left")

# get weekly roster data
agg_WR.week = agg_WR.week.astype(int)
agg_WR_final = agg_WR.drop('receiver_player_name',axis=1).merge(rosters.drop('game_id', axis=1), on=['week', 'player_id'], how="left")
agg_WR_final = agg_WR_final.merge(id_map[['gsis_id', 'birthdate', 'age',
                                        'draft_year', 'draft_round', 'draft_pick', 'draft_ovr',
                                        'twitter_username', 'height', 'weight', 'college']].rename({'gsis_id':"player_id"},axis=1),
                                            on = "player_id", how="left")

In [19]:
# getting snap percentages
temp = agg_WR_final.merge(team_snaps, on=['game_id', 'team'], how='left').copy()
temp['snap_percentage_1'] = temp['snap_count_1']/temp["tsnap_count_1"]
temp['snap_percentage_2'] = temp['snap_count_2']/temp["tsnap_count_2"]
temp['snap_percentage_3'] = temp['snap_count_3']/temp["tsnap_count_3"]
temp['snap_percentage_4'] = temp['snap_count_4']/temp["tsnap_count_4"]
temp['snap_percentage'] = temp['total_relevant_snaps']/temp["ttotal_relevant_snaps"]

In [20]:
temp = temp.merge(team_targets_game, on=['game_id', 'team'], how='left').copy()
temp['target_share_1'] = temp['targets_1']/temp["ttargets_1"]
temp['target_share_2'] = temp['targets_2']/temp["ttargets_2"]
temp['target_share_3'] = temp['targets_3']/temp["ttargets_3"]
temp['target_share_4'] = temp['targets_4']/temp["ttargets_4"]
temp['target_share'] = temp['total_targets']/temp["ttotal_targets"]

In [21]:
temp = temp.merge(team_air_yards_game, on=['game_id', 'team'], how='left').copy()
temp['air_yards_share'] = temp['air_yards']/temp["team_air_yards"]

In [22]:
agg_WR_final = temp.drop(
    [
        "tsnap_count_1",
        "tsnap_count_2",
        "tsnap_count_3",
        "tsnap_count_4",
        "ttotal_relevant_snaps",
        "ttargets_1",
        "ttargets_2",
        "ttargets_3",
        "ttargets_4",
        "ttotal_targets",
        'team_air_yards'
    ],
    axis=1,
)
agg_WR_final = agg_WR_final.merge(
    depth_charts[["game_id", "gsis_id", "depth_team"]],
    left_on=["game_id", "player_id"],
    right_on=["game_id", "gsis_id"],
    how="left",
).drop("gsis_id", axis=1)

agg_WR_final['wopr'] = 1.5*agg_WR_final['target_share'] + 0.7*agg_WR_final['air_yards_share']

agg_WR_final.head()

,player_id,game_id,receiving_yards,avg_yac,receptions,receiving_touchdowns,week,targets_1,targets_2,targets_3,...,snap_percentage_4,snap_percentage,target_share_1,target_share_2,target_share_3,target_share_4,target_share,air_yards_share,depth_team,wopr
0,00-0024243,2022_05_NYG_GB,2.0,0.0,1.0,1.0,5,0.0,1.0,0.0,...,1.000000,0.507692,0.000000,0.062500,0.0,0.000000,0.024390,0.006873,1,0.041396
1,00-0024243,2022_09_GB_DET,19.0,5.0,1.0,0.0,9,0.0,0.0,1.0,...,0.500000,0.424658,0.000000,0.000000,0.1,0.000000,0.022727,0.032634,1,0.056935
2,00-0024243,2022_10_DAL_GB,-1.0,0.0,1.0,0.0,10,1.0,0.0,0.0,...,0.333333,0.575758,0.125000,0.000000,0.0,0.000000,0.045455,-0.004425,1,0.065084
3,00-0024243,2022_15_LA_GB,14.0,7.0,1.0,0.0,15,0.0,1.0,0.0,...,0.500000,0.408451,0.000000,0.083333,0.0,NaN,0.030303,0.053030,1,0.082576
4,00-0024243,2022_16_GB_MIA,32.0,1.0,2.0,1.0,16,1.0,0.0,0.0,...,0.750000,0.455882,0.071429,0.000000,0.0,0.333333,0.050000,0.091185,1,0.138830


In [23]:
agg_WR_final[~agg_WR_final.team.isna()].to_excel(f'./data/final_gbg_receiver_data_{year}.xlsx', index=False)

In [24]:
agg_WR_final.groupby(['player_id', 'player_name', 'team'])['receiving_yards'].sum().reset_index().sort_values('receiving_yards', ascending = False).iloc[:20]

,player_id,player_name,team,receiving_yards
460,00-0036322,Justin Jefferson,MIN,1856.0
140,00-0033040,Tyreek Hill,MIA,1779.0
382,00-0035676,A.J. Brown,PHI,1674.0
44,00-0030506,Travis Kelce,KC,1620.0
78,00-0031588,Stefon Diggs,BUF,1578.0
469,00-0036358,CeeDee Lamb,DAL,1544.0
64,00-0031381,Davante Adams,LV,1516.0
555,00-0036912,DeVonta Smith,PHI,1494.0
510,00-0036613,Jaylen Waddle,MIA,1400.0
553,00-0036900,Ja'Marr Chase,CIN,1266.0


### Some bookeeping

1. Before we average out over games, we need to be sure that players who were not involved with the passing game on any particular week but were also not injured have those games included in averages etc. 

In [25]:
# gtfp.drop('game_id', axis=1).groupby(['receiver_player_id', 'receiver_player_name'])['garbage_time_fpoints'].mean().reset_index().sort_values('garbage_time_fpoints', ascending=False)

In [26]:
# TODO: Need to fill in nulls for players who got nothing in any particular game.
# NOTE: If players were injured then that game shouldn't be considered for them
# per_game_stats = pd.concat([rec_yards_game.set_index(['receiver_player_id', 'receiver_player_name', 'game_id']),
#     targets_game.set_index(['receiver_player_id', 'receiver_player_name', 'game_id']),
#     average_YAC_game.set_index(['receiver_player_id', 'receiver_player_name', 'game_id']),
#     average_air_yards_game.set_index(['receiver_player_id', 'receiver_player_name', 'game_id']),
#     receptions_game.set_index(['receiver_player_id', 'receiver_player_name', 'game_id']),
#     max_air_yards_game.set_index(['receiver_player_id', 'receiver_player_name', 'game_id'])], axis=1).reset_index()
# per_game_stats.head()



In [27]:
error_list = {}
for idx, row in pbp_data[['receiver_player_id', 'receiver_player_name']].dropna().iterrows():
    id_map_name = id_map[id_map['gsis_id'] == row['receiver_player_id']]['name']

    if len(id_map_name) > 1:
        print(f"More than 1 match for pbp name: {row['receiver_player_name']}, id: {row['receiver_player_id']}\nfrom id_map name: {id_map_name}\n")
    elif len(id_map_name) ==0:
        print(f"No match for pbp name: {row['receiver_player_name']}, id: {row['receiver_player_id']}\n")
    elif id_map_name.values[0].split(' ')[-1] != row['receiver_player_name'].split('.')[-1] and row['receiver_player_id'] not in error_list.keys():
        error_list.update({row['receiver_player_id']:f"{row['receiver_player_name']}/{id_map_name}"})
        print(f"from pbp name: {row['receiver_player_name']}, id: {row['receiver_player_id']}\nfrom id_map name: {id_map_name}\n")

from pbp name: R.Anderson, id: 00-0032688
from id_map name: 3085    Robbie Chosen
Name: name, dtype: object

from pbp name: I.Smith, id: 00-0034970
from id_map name: 1571    Irv Smith Jr.
Name: name, dtype: object

from pbp name: A.St. Brown, id: 00-0036963
from id_map name: 785    Amon-Ra St. Brown
Name: name, dtype: object

from pbp name: E.St. Brown, id: 00-0034279
from id_map name: 1979    Equanimeous St. Brown
Name: name, dtype: object

from pbp name: K.Walker, id: 00-0038134
from id_map name: 356    Kenneth Walker III
Name: name, dtype: object

No match for pbp name: T.Conner, id: 00-0037348

No match for pbp name: T.Conner, id: 00-0037348

No match for pbp name: T.Conner, id: 00-0037348

No match for pbp name: M.Woods, id: 00-0037300

No match for pbp name: M.Woods, id: 00-0037300

No match for pbp name: M.Woods, id: 00-0037300

No match for pbp name: M.Woods, id: 00-0037300

No match for pbp name: D.Bakhtiari, id: 00-0030074

No match for pbp name: M.Woods, id: 00-0037300

No m

In [28]:
id_map[id_map.gsis_id.isin(agg_WR_final[agg_WR_final.player_name.isnull()].player_id.unique().tolist())][['gsis_id', 'name']]

,gsis_id,name


# Running Backs

In [29]:
agg_RB = pbp_data[(pbp_data['play_type'] == 'run')].groupby(['rusher_player_id', 'rusher_player_name', 'game_id']).agg(
    rushing_yards = ('rushing_yards', 'sum'),
    avg_ypc = ('rushing_yards', 'mean'),
    carries = ('play_type', 'count'),
    rushing_touchdowns = ('touchdown', 'sum'),
    week = ('week', 'first')
).reset_index()
agg_RB

,rusher_player_id,rusher_player_name,game_id,rushing_yards,avg_ypc,carries,rushing_touchdowns,week
0,00-0019596,T.Brady,2022_02_TB_NO,0.0,0.000000,1,0.0,2
1,00-0019596,T.Brady,2022_06_TB_PIT,0.0,0.000000,1,0.0,6
2,00-0019596,T.Brady,2022_07_TB_CAR,1.0,1.000000,1,0.0,7
3,00-0019596,T.Brady,2022_08_BAL_TB,1.0,1.000000,1,0.0,8
4,00-0019596,T.Brady,2022_12_TB_CLE,2.0,2.000000,1,0.0,12
...,...,...,...,...,...,...,...,...
2332,00-0038134,K.Walker,2022_15_SF_SEA,47.0,3.916667,12,0.0,15
2333,00-0038134,K.Walker,2022_16_SEA_KC,107.0,4.115385,26,0.0,16
2334,00-0038134,K.Walker,2022_17_NYJ_SEA,133.0,5.782609,23,0.0,17
2335,00-0038134,K.Walker,2022_18_LA_SEA,114.0,3.931035,29,0.0,18


In [30]:
# FIXME: For players who were not injured, but did not receive any of the above stats, I need to insert rows here.

for _, player in rosters[rosters['position'].isin(['WR', 'RB', 'FB', 'TE', 'QB'])].iterrows():
    if agg_RB[(agg_RB.rusher_player_id.isin([player.player_id])) & (agg_RB.week.isin([player.week]))].empty:
        agg_RB = pd.concat([agg_RB, pd.DataFrame([[player.player_id, None, player.game_id, 0, 0, 0, 0, player.week]], columns = agg_RB.columns)], ignore_index=True)

In [31]:
# importing next gen stats data
ngs_rush = nfl.import_ngs_data(stat_type = 'rushing', years = [year]).drop_duplicates()

# pro football reference stats
pfr_rush = nfl.import_weekly_pfr(s_type = "rush", years = [year]).drop_duplicates()

In [32]:
# weekly_rush = nfl.import_weekly_data(years = [year], downcast=True, columns=["player_id", 'rushing_fumbles', 'rushing_fumbles_lost','week', 'opponent_team'])
# weekly_rush = weekly_rush.merge(game_id_map, left_on=['week', 'opponent_team'], right_on = ['week', 'team_abbr'])
# weekly_rush.head()

In [33]:
# grab game_id for ngs/weekly datasets
ngs_rush = ngs_rush.merge(game_id_map, left_on=['week', 'team_abbr'], right_on = ['week', 'team_abbr']).rename({'player_gsis_id':'player_id'},axis=1)
pfr_rush = pfr_rush.merge(id_map[['pfr_id', 'gsis_id']], left_on='pfr_player_id', right_on = 'pfr_id').rename({'gsis_id':'player_id'},axis=1)

In [34]:
# rushing fumbles
rush_fumbles = pbp_data[
    (pbp_data['play_type'] == 'run')  & (pbp_data["fumble"] == 1)
].groupby(["rusher_player_id", "game_id"])['fumble'].sum().reset_index().rename({'fumble':'rushing_fumbles'}, axis=1)

# rushing fumbles lost
rush_fumbles_lost = pbp_data[
    (pbp_data['play_type'] == 'run')  & (pbp_data["fumble_lost"] == 1)
].groupby(["rusher_player_id", "game_id"])['fumble_lost'].sum().reset_index().rename({'fumble_lost':'rushing_fumbles_lost'}, axis=1)

In [35]:
# number of targets per game
opps_game = (
    pbp_data[(pbp_data['play_type'] == 'run') & (pbp_data["down"].isin([1, 2, 3, 4]))]
    .groupby(["rusher_player_id", "game_id", "down"])['play_type']
    .count()
    .reset_index()
    .rename({'play_type': "opps"}, axis=1)
)
opps_game = (
    opps_game.pivot(
        index=["game_id", "rusher_player_id"], columns=["down"], values="opps"
    )
    .reset_index()
    .fillna(0)
    .rename({1.0: "opps_1", 2.0: "opps_2", 3.0: "opps_3", 4.0: "opps_4"}, axis=1)
)
opps_game["total_opps"] = (
    opps_game["opps_1"]
    + opps_game["opps_2"]
    + opps_game["opps_3"]
    + opps_game["opps_4"]
)
# num_snaps is a bit more complicated. For each player, we need to count the number of times their player ID appeared in the offensive snaps,
# then we can count the number of offensive snaps and get the percentage from that.
# snaps = nfl.import_snap_counts(years = [year]).merge(id_map, left_on='pfr_player_id', right_on = 'pfr_id')

# redzone targets
redzone_opps_game = (
    pbp_data[
        (pbp_data['play_type'] == 'run')
        & (pbp_data["yardline_100"] <= 20)
    ]
    .groupby(["rusher_player_id", "game_id"])['play_type']
    .count()
    .reset_index()
    .rename({"play_type": "rz_opps"}, axis=1)
)

# garbage time points?
# fantasy points scored when the point differential is more than 2 scores with 10 or less minutes left in the game
# TODO: Address the definition here. Should the point total be higher? What if they're down by 4 scores with a quarter left?
grouped_rush = pbp_data[
(
    (
        (pbp_data['play_type'] == 'run')
        & (pbp_data["score_differential"] <= -28)
        & (pbp_data["game_seconds_remaining"] <= 15 * 60)
    )
    | (
        (pbp_data['play_type'] == 'run')
        & (pbp_data["score_differential"] <= -21)
        & (pbp_data["game_seconds_remaining"] <= 10 * 60)
    )
    | (
        (pbp_data['play_type'] == 'run')
        & (pbp_data["score_differential"] <= -14)
        & (pbp_data["game_seconds_remaining"] <= 3 * 60)
    )  # ((pbp_data['complete_pass'] == 1) & (pbp_data['score_differential'] < -10) & (pbp_data['game_seconds_remaining'] <= 2*60)) \
)
].groupby(["rusher_player_id", "game_id"])


def get_gtfp_rush(x):
    d = {}
    d["garbage_time_fpoints"] = (
        0.1 * x["yards_gained"] + 6 * x["touchdown"] + 2 * x["is_two_point_conversion"]
    ).sum()
    return pd.Series(d, index=["garbage_time_fpoints"])


gtfp_rush = grouped_rush.apply(get_gtfp_rush).reset_index()

# fantasy points per game (PPR)
grouped2_rush = pbp_data[
    (pbp_data['play_type'] == 'run') 
].groupby(["rusher_player_id", "game_id"])


def get_fp_rush(x):
    d = {}
    d["rushing_fpoints"] = (
        0.1 * x["yards_gained"]
        + 6 * x["touchdown"]
        + 2 * x["is_two_point_conversion"]
    ).sum()
    return pd.Series(d, index=["rushing_fpoints"])


rfp_rush = grouped2_rush.apply(get_fp_rush).reset_index()

In [36]:
rfp_rush.sort_values('rushing_fpoints', ascending=False)

,rusher_player_id,game_id,rushing_fpoints
724,00-0033897,2022_09_CAR_CIN,39.300000
1348,00-0035700,2022_12_LV_SEA,34.900001
342,00-0032764,2022_08_TEN_HOU,33.900000
1646,00-0036389,2022_22_KC_PHI,33.200000
1343,00-0035700,2022_07_HOU_LV,32.300000
...,...,...,...
834,00-0034160,2022_05_SEA_NO,-0.800000
1388,00-0035719,2022_21_SF_PHI,-0.900000
255,00-0031763,2022_15_TEN_LAC,-1.000000
2201,00-0037741,2022_01_JAX_WAS,-1.000000


In [37]:
# merge all this together
agg_RB = agg_RB.merge(opps_game, on=['game_id', 'rusher_player_id'], how='outer')
agg_RB = agg_RB.merge(redzone_opps_game, on=['game_id', 'rusher_player_id'], how='outer')
agg_RB = agg_RB.merge(gtfp_rush, on=['game_id', 'rusher_player_id'], how='outer')
agg_RB = agg_RB.merge(rfp_rush, on=['game_id', 'rusher_player_id'], how='outer')
agg_RB = agg_RB.merge(rush_fumbles, on=['game_id', 'rusher_player_id'], how='outer')
agg_RB = agg_RB.merge(rush_fumbles_lost, on=['game_id', 'rusher_player_id'], how='outer')
agg_RB = agg_RB.merge(player_snaps.rename({"receiver_player_id":"rusher_player_id"},axis=1), on=['game_id', "rusher_player_id"], how='left').fillna(0).rename({'rusher_player_id':'player_id'},axis=1)

agg_RB.head()

,player_id,rusher_player_name,game_id,rushing_yards,avg_ypc,carries,rushing_touchdowns,week,opps_1,opps_2,...,rz_opps,garbage_time_fpoints,rushing_fpoints,rushing_fumbles,rushing_fumbles_lost,snap_count_1,snap_count_2,snap_count_3,snap_count_4,total_relevant_snaps
0,00-0019596,T.Brady,2022_02_TB_NO,0.0,0.0,1,0.0,2,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,26.0,23.0,19.0,1.0,69.0
1,00-0019596,T.Brady,2022_06_TB_PIT,0.0,0.0,1,0.0,6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.0,25.0,14.0,2.0,74.0
2,00-0019596,T.Brady,2022_07_TB_CAR,1.0,1.0,1,0.0,7,0.0,1.0,...,1.0,0.0,0.1,0.0,0.0,28.0,24.0,14.0,3.0,69.0
3,00-0019596,T.Brady,2022_08_BAL_TB,1.0,1.0,1,0.0,8,0.0,0.0,...,1.0,0.0,0.1,0.0,0.0,28.0,23.0,14.0,2.0,67.0
4,00-0019596,T.Brady,2022_12_TB_CLE,2.0,2.0,1,0.0,12,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,28.0,24.0,19.0,2.0,73.0


In [38]:
len(agg_RB)

7908

In [39]:
# grabbing stats from other datasets
# print(f"In-house dataset-length: {len(agg_RB)}")
# agg_RB = agg_RB.merge(
#     weekly_rush[['rushing_fumbles', 'rushing_fumbles_lost',
#             "game_id", 
#             "player_id"]],
#     on=["game_id", "player_id"],
#     how="left",
# )
# print(f"Post Weekly merge: {len(agg_RB)}")

agg_RB = agg_RB.merge(
    pfr_rush[['game_id',
        'player_id',
        'rushing_yards_before_contact', 
        'rushing_yards_before_contact_avg',
        'rushing_yards_after_contact', 
        'rushing_yards_after_contact_avg',
        'rushing_broken_tackles']],
    on=["game_id", "player_id"],
    how="left",
)
print(f"Post PFR merge: {len(agg_RB)}")

agg_RB = agg_RB.merge(
    ngs_rush[['game_id',
        'player_id',
        'efficiency',
        'percent_attempts_gte_eight_defenders', 
        'avg_time_to_los',
        'expected_rush_yards',
        'rush_yards_over_expected',
        'rush_yards_over_expected_per_att',
        'rush_pct_over_expected',
        'rush_touchdowns']],
    on=["game_id", "player_id"],
    how="left",
)
print(f"Post NGS merge: {len(agg_RB)}")

# agg_RB_final = agg_RB.merge(id_map[['gsis_id', 'name', 'position', 'team', 'birthdate', 'age',
#                                                             'draft_year', 'draft_round', 'draft_pick', 'draft_ovr',
#                                                             'twitter_username', 'height', 'weight', 'college']].rename({'gsis_id':"player_id"},axis=1),
#                                                               on = "player_id", how="left")
agg_RB.week = agg_RB.week.astype(int)
agg_RB_final = agg_RB.drop('rusher_player_name',axis=1).merge(rosters.drop('game_id', axis=1), on=['week', 'player_id'], how="left")
agg_RB_final = agg_RB_final.merge(id_map[['gsis_id', 'birthdate', 'age',
                                        'draft_year', 'draft_round', 'draft_pick', 'draft_ovr',
                                        'twitter_username', 'height', 'weight', 'college']].rename({'gsis_id':"player_id"},axis=1),
                                            on = "player_id", how="left")

Post PFR merge: 7908
Post NGS merge: 7908


In [40]:
id_map[id_map.gsis_id.isin(agg_RB[agg_RB.week.isna()].player_id.unique().tolist())][['gsis_id', 'name']]

,gsis_id,name


In [41]:
agg_RB_final.head()

,player_id,game_id,rushing_yards,avg_ypc,carries,rushing_touchdowns,week,opps_1,opps_2,opps_3,...,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college
0,00-0019596,2022_02_TB_NO,0.0,0.0,1,0.0,2,0.0,0.0,1.0,...,1977-08-03,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan
1,00-0019596,2022_06_TB_PIT,0.0,0.0,1,0.0,6,0.0,0.0,1.0,...,1977-08-03,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan
2,00-0019596,2022_07_TB_CAR,1.0,1.0,1,0.0,7,0.0,1.0,0.0,...,1977-08-03,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan
3,00-0019596,2022_08_BAL_TB,1.0,1.0,1,0.0,8,0.0,0.0,1.0,...,1977-08-03,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan
4,00-0019596,2022_12_TB_CLE,2.0,2.0,1,0.0,12,0.0,0.0,0.0,...,1977-08-03,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan


In [42]:
agg_RB_final = agg_RB_final.merge(depth_charts[['game_id', 'gsis_id', 'depth_team']], left_on = ['game_id', 'player_id'], right_on = ['game_id', 'gsis_id']).drop('gsis_id', axis=1)

In [43]:
agg_RB_final.head()

,player_id,game_id,rushing_yards,avg_ypc,carries,rushing_touchdowns,week,opps_1,opps_2,opps_3,...,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,depth_team
0,00-0019596,2022_02_TB_NO,0.0,0.0,1,0.0,2,0.0,0.0,1.0,...,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,1
1,00-0019596,2022_06_TB_PIT,0.0,0.0,1,0.0,6,0.0,0.0,1.0,...,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,1
2,00-0019596,2022_07_TB_CAR,1.0,1.0,1,0.0,7,0.0,1.0,0.0,...,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,1
3,00-0019596,2022_08_BAL_TB,1.0,1.0,1,0.0,8,0.0,0.0,1.0,...,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,1
4,00-0019596,2022_12_TB_CLE,2.0,2.0,1,0.0,12,0.0,0.0,0.0,...,46.3,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,1


In [44]:
agg_RB_final[~agg_RB_final.team.isna()].to_excel(f'./data/final_gbg_rusher_data_{year}.xlsx', index=False)

In [45]:
# should you prioritize young receivers? e.g. do young receivers actually perform better for fantasy than old receivers?
# should we prioritize fast receivers? e.g. do fast receivers actually perform better for fantasy than slow ones?
# what is the most important factor in receiver performance?
# to answer all these we probably need to come up with a normalized fantasy performance. Relative to draft round. Maybe an ROI?


# is there a meaningful increase in fantasy performance when the receiver above you goes out on injury?
# Are there statistical tiers for WR's by the end of the year? i.e. their mean fantasy performance is significantly  (statistically) different from other receivers?
# can we predict fantasy performance?

# QBs

Let's grab some stats for QB's. For now, I won't be doing any prediction on QB's, I just want to feed in some of those stats to my model

In [46]:
agg_WR_final[agg_WR_final.depth_team.isna()][['game_id', 'player_id', 'player_name']]

,game_id,player_id,player_name
6,2022_09_BAL_NO,00-0026189,DeSean Jackson
7,2022_12_BAL_JAX,00-0026189,DeSean Jackson
8,2022_13_DEN_BAL,00-0026189,DeSean Jackson
9,2022_14_BAL_PIT,00-0026189,DeSean Jackson
49,2022_11_TEN_GB,00-0028002,Randall Cobb
...,...,...,...
7923,2022_13_JAX_DET,00-0037240,Jameson Williams
7930,2022_16_DET_CAR,00-0033095,Garrett Griffin
7943,2022_09_GB_DET,00-0034052,Brandon Zylstra
7947,2022_09_GB_DET,00-0037581,Stanley Berryhill


In [47]:
test = nfl.import_depth_charts(years = [year])
test.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
0,2022,ATL,17,REG,1,Williams,Avery,Avery,Special Teams,00-0036950,26,RB,WIL145932,PR,Avery Williams
1,2022,ATL,17,REG,1,Williams,Avery,Avery,Special Teams,00-0036950,26,RB,WIL145932,KR,Avery Williams
2,2022,ATL,17,REG,2,Patterson,Cordarrelle,Cordarrelle,Special Teams,00-0030578,84,RB,PAT387075,KR,Cordarrelle Patterson
3,2022,ATL,17,REG,1,McCullough,Liam,Liam,Special Teams,00-0036082,49,LS,MCC769873,LS,Liam McCullough
4,2022,ATL,17,REG,1,Pinion,Bradley,Bradley,Special Teams,00-0032065,13,P,PIN501016,H,Bradley Pinion


In [48]:
test[test.week == 11]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
266,2022,ATL,11,REG,2,Ifedi,Germain,Germain,Offense,00-0032384,74,T,IFE136317,RT,Germain Ifedi
267,2022,ATL,11,REG,2,Williams,Avery,Avery,Offense,00-0036950,26,RB,WIL145932,RB,Avery Williams
268,2022,ATL,11,REG,1,Ogundeji,Adetokunbo,Adetokunbo,Defense,00-0036949,92,OLB,OGU424705,OLB,Adetokunbo Ogundeji
269,2022,ATL,11,REG,1,Jarrett,Grady,Grady,Defense,00-0031583,97,DT,JAR254349,DL,Grady Jarrett
270,2022,ATL,11,REG,1,Anderson,Abdullah,Abdullah,Defense,00-0034587,92,DE,AND036084,DL,Abdullah Anderson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37327,2022,HOU,11,REG,3,Johnson,Tyron,Tyron,Offense,00-0035457,18,WR,JOH711246,WR,Tyron Johnson
37328,2022,HOU,11,REG,3,Akins,Jordan,Jordan,Offense,00-0034364,84,TE,AKI501776,TE,Jordan Akins
37329,2022,HOU,11,REG,1,Tunsil,Laremy,Laremy,Offense,00-0032380,78,T,TUN565442,LT,Laremy Tunsil
37330,2022,HOU,11,REG,2,Heck,Charles,Charlie,Offense,00-0036195,67,T,HEC157579,LT,Charlie Heck


In [49]:
agg_WR_final[agg_WR_final.player_name == "Drake London"][['week', 'player_name', 'player_id', 'depth_team']]

,week,player_name,player_id,depth_team
3604,1,Drake London,00-0037238,3
3605,2,Drake London,00-0037238,3
3606,3,Drake London,00-0037238,1
3607,4,Drake London,00-0037238,1
3608,5,Drake London,00-0037238,1
3609,6,Drake London,00-0037238,1
3610,7,Drake London,00-0037238,1
3611,8,Drake London,00-0037238,1
3612,9,Drake London,00-0037238,1
3613,10,Drake London,00-0037238,1


# Final Merge

To get the final data, we will merge on the player id

In [50]:
# common columns
[x for x in agg_WR_final.columns if x in agg_RB_final.columns]

['player_id',
 'game_id',
 'week',
 'garbage_time_fpoints',
 'snap_count_1',
 'snap_count_2',
 'snap_count_3',
 'snap_count_4',
 'total_relevant_snaps',
 'position',
 'player_name',
 'team',
 'birthdate',
 'age',
 'draft_year',
 'draft_round',
 'draft_pick',
 'draft_ovr',
 'twitter_username',
 'height',
 'weight',
 'college',
 'depth_team']

In [51]:
agg_final = agg_WR_final.merge(agg_RB_final, how='outer', on =['player_id',
 'game_id',
 'week',
 'snap_count_1',
 'snap_count_2',
 'snap_count_3',
 'snap_count_4',
 'total_relevant_snaps',
 'position',
 'player_name',
 'team',
 'birthdate',
 'age',
 'draft_year',
 'draft_round',
 'draft_pick',
 'draft_ovr',
 'twitter_username',
 'height',
 'weight',
 'college',
 'depth_team'])

In [52]:
print(f"The length of the original WR dataset is {len(agg_WR_final)}. RB dataset: {len(agg_RB_final)}. Merged dataset: {len(agg_final)}")

The length of the original WR dataset is 7956. RB dataset: 7258. Merged dataset: 8029


In [53]:
agg_final['total_garbage_fpoints'] = agg_final['garbage_time_fpoints_x'].fillna(0) + agg_final['garbage_time_fpoints_y'].fillna(0)
agg_final.drop(['garbage_time_fpoints_x','garbage_time_fpoints_y'], axis=1, inplace=True)

agg_final['fpoints'] = agg_final['receiving_fpoints'].fillna(0) + agg_final['rushing_fpoints'].fillna(0)

In [54]:
agg_final.dropna(subset=['team'], inplace=True)

In [55]:
agg_final = agg_final[[
    'player_id', 'game_id', 'week', 'position', 'player_name', 'team', 'fpoints', 'receiving_yards', 'avg_yac',
       'avg_depth_of_target', 'receptions', 'air_yards', 'unrealized_air_yards', 'max_target_depth',
       'receiving_touchdowns', 'targets_1', 'targets_2', 'targets_3',
       'targets_4', 'total_targets', 'rz_targets', 'receiving_fpoints',
       'snap_count_1', 'snap_count_2', 'snap_count_3', 'snap_count_4','total_relevant_snaps',
       'snap_percentage_1', 'snap_percentage_2',
       'snap_percentage_3', 'snap_percentage_4', 'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions',
       'racr', 'target_share', 'air_yards_share', 'wopr',
       'receiving_broken_tackles', 'receiving_drop', 'receiving_drop_pct',
       'receiving_int', 'receiving_rat', 'avg_cushion', 'avg_separation',
         'depth_team', 'rushing_yards',
       'avg_ypc', 'carries', 'rushing_touchdowns', 'opps_1', 'opps_2',
       'opps_3', 'opps_4', 'total_opps', 'rz_opps', 'rushing_fpoints',
       'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_yards_before_contact', 'rushing_yards_before_contact_avg',
       'rushing_yards_after_contact', 'rushing_yards_after_contact_avg',
       'rushing_broken_tackles', 'efficiency',
       'percent_attempts_gte_eight_defenders', 'avg_time_to_los',
       'expected_rush_yards', 'rush_yards_over_expected',
       'rush_yards_over_expected_per_att', 'rush_pct_over_expected',
       'rush_touchdowns', 'total_garbage_fpoints','birthdate', 'age', 'draft_year',
       'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 'height',
       'weight', 'college'
]]

In [56]:
agg_final

,player_id,game_id,week,position,player_name,team,fpoints,receiving_yards,avg_yac,avg_depth_of_target,...,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college
0,00-0024243,2022_05_NYG_GB,5,TE,Marcedes Lewis,GB,7.2,2.0,0.0,2.0,...,1984-05-19,39.5,2006.0,1.0,28.0,28.0,MarcedesLewis89,78.0,265.0,UCLA
1,00-0024243,2022_09_GB_DET,9,TE,Marcedes Lewis,GB,2.9,19.0,5.0,14.0,...,1984-05-19,39.5,2006.0,1.0,28.0,28.0,MarcedesLewis89,78.0,265.0,UCLA
2,00-0024243,2022_10_DAL_GB,10,TE,Marcedes Lewis,GB,0.9,-1.0,0.0,-1.0,...,1984-05-19,39.5,2006.0,1.0,28.0,28.0,MarcedesLewis89,78.0,265.0,UCLA
3,00-0024243,2022_15_LA_GB,15,TE,Marcedes Lewis,GB,2.4,14.0,7.0,7.0,...,1984-05-19,39.5,2006.0,1.0,28.0,28.0,MarcedesLewis89,78.0,265.0,UCLA
4,00-0024243,2022_16_GB_MIA,16,TE,Marcedes Lewis,GB,11.2,32.0,1.0,15.0,...,1984-05-19,39.5,2006.0,1.0,28.0,28.0,MarcedesLewis89,78.0,265.0,UCLA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,00-0034440,2022_08_MIA_DET,8,RB,Justin Jackson,DET,0.0,0.0,0.0,0.0,...,1995-04-22,28.6,2018.0,7.0,33.0,251.0,NaN,72.0,199.0,Northwestern
8022,00-0035544,2022_08_MIA_DET,8,WR,Tom Kennedy,DET,0.0,0.0,0.0,0.0,...,1996-07-29,27.3,2019.0,NaN,NaN,NaN,NaN,70.0,194.0,Bryant
8023,00-0035567,2022_08_MIA_DET,8,RB,Craig Reynolds,DET,0.7,0.0,0.0,4.0,...,1996-06-15,27.4,2019.0,NaN,NaN,NaN,NaN,71.0,216.0,Kutztown
8024,00-0037581,2022_08_MIA_DET,8,WR,Stanley Berryhill,DET,0.0,0.0,0.0,0.0,...,1998-06-09,25.5,2022.0,NaN,NaN,NaN,NaN,71.0,190.0,Arizona


In [57]:
agg_final[~agg_final.team.isna()].to_excel(f'./data/final_gbg_rr_{year}.xlsx', index=False)

In [58]:
toy = pd.DataFrame([
    {"receiver_player_id":1, "game_id":"A", "pass_attempt":1, "air_yards":10},
    {"receiver_player_id":1, "game_id":"A", "pass_attempt":1, "air_yards":20},
    {"receiver_player_id":1, "game_id":"A", "pass_attempt":1, "air_yards":None},
    {"receiver_player_id":2, "game_id":"A", "pass_attempt":1, "air_yards":1},
    {"receiver_player_id":2, "game_id":"A", "pass_attempt":1, "air_yards":1},
    {"receiver_player_id":2, "game_id":"A", "pass_attempt":1, "air_yards":2},
    {"receiver_player_id":1, "game_id":"B", "pass_attempt":1, "air_yards":65},
    {"receiver_player_id":1, "game_id":"B", "pass_attempt":1, "air_yards":None},
    {"receiver_player_id":2, "game_id":"B", "pass_attempt":1, "air_yards":2},
    {"receiver_player_id":3, "game_id":"B", "pass_attempt":0, "air_yards":2},
])
ayg_toy = (
    toy[(toy["pass_attempt"] == 1)]
    .groupby(["receiver_player_id", "game_id"])
    .agg(
        avg_depth_of_target = ('air_yards', 'mean'),
        air_yards = ('air_yards', 'sum'),
        max_target_depth = ('air_yards', 'max')
    )
    .reset_index()
)
ayg_toy 

,receiver_player_id,game_id,avg_depth_of_target,air_yards,max_target_depth
0,1,A,15.000000,30.0,20.0
1,1,B,65.000000,65.0,65.0
2,2,A,1.333333,4.0,2.0
3,2,B,2.000000,2.0,2.0
